In [5]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import classification_report, accuracy_score, roc_auc_score
from xgboost import XGBClassifier

df = pd.read_excel('fema_floods.xlsx')

label_encoder = LabelEncoder()

df['lgb_selfid'] = label_encoder.fit_transform(df['lgb_selfid'])
df['cfld_awareness'] = label_encoder.fit_transform(df['cfld_awareness'])

df['cfld_confidence'] = label_encoder.fit_transform(df['cfld_confidence'])

X = df[['lgb_selfid', 'cfld_awareness']]  #features
y = df['cfld_confidence'] #target

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

#scaling just to be safe!
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

#https://xgboost.readthedocs.io/en/stable/tutorials/index.html
#https://www.kaggle.com/discussions/general/237096
model = XGBClassifier(random_state=42)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:")
print(classification_report(y_test, y_pred))

Accuracy: 0.3235294117647059

Classification Report:
              precision    recall  f1-score   support

           0       0.57      0.27      0.36        15
           1       0.35      0.57      0.43        23
           2       0.17      0.04      0.06        26
           3       0.00      0.00      0.00         3
           4       0.00      0.00      0.00        10
           5       0.29      0.60      0.39        25

    accuracy                           0.32       102
   macro avg       0.23      0.25      0.21       102
weighted avg       0.28      0.32      0.26       102



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [7]:
misclassified_samples = X_test[y_pred != y_test]
misclassified_ids = df.iloc[y_test.index][y_pred != y_test]['id']

misclassified_samples = pd.DataFrame(misclassified_samples, columns=['lgb_selfid', 'cfld_awareness'], index=y_test[y_pred != y_test].index) #https://datascience.stackexchange.com/questions/58961/sklearn-accuracy-scorey-test-y-predict-vs-np-meany-predict-y-test
misclassified_samples = pd.concat([misclassified_samples, misclassified_ids], axis=1)
misclassified_samples.head()

,lgb_selfid,cfld_awareness,id
492,-0.144943,-0.568200,493
72,-0.144943,1.052370,73
453,-0.144943,-0.568200,454
76,-0.144943,-2.188771,77
472,-0.144943,-0.568200,473
